# Quickmodel
A simple command project I'm putting together. Goal: User runs the file with a specific search term, and the program returns a pickled ML model that is trained to recognize images of the search term. 

In [19]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.optimizers import SGD
img_dir = os.listdir('../data/')
img_count = len(img_dir)
type(img_dir)

list

First, let's generate our dataset from our existing files. Luckily, ``keras`` has a nice way of implementing this from their Image library.

In [12]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1./255, 
    validation_split=0.3
)

IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE=20
STEPS_PER_EPOCH = np.ceil(img_count/BATCH_SIZE)

train_generator = image_generator.flow_from_directory(directory='../data',
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = ['Eggs', 'NOT-Eggs'], #should come from script parameter
                                                     subset='training',
                                                     class_mode='binary') 
validation_generator = image_generator.flow_from_directory(directory='../data',
                                                          target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                          classes= ['Eggs', 'NOT-Eggs'],
                                                          subset='validation',
                                                          class_mode='binary')

Found 117 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


Now that we have our dataset, we need to construct a model and train it.

In [33]:

model = keras.models.Sequential([
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 2, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])


Now, let's train and test our model. Keep in mind, the dataset is quite small so accuracy might be low. We will probably need to generate more augmented data. 

In [38]:
from tensorflow.keras.callbacks import EarlyStopping

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = 30,
    callbacks=[EarlyStopping(monitor='val_loss', patience=4)],
)

Epoch 1/100
5/5 [==============================] - 20s 4s/step - loss: 0.5883 - accuracy: 0.6779 - val_loss: 0.5696 - val_accuracy: 0.6400
Epoch 2/100
5/5 [==============================] - 17s 3s/step - loss: 0.5414 - accuracy: 0.7517 - val_loss: 0.5544 - val_accuracy: 0.6600
Epoch 3/100
5/5 [==============================] - 16s 3s/step - loss: 0.5461 - accuracy: 0.7246 - val_loss: 0.5481 - val_accuracy: 0.7200
Epoch 4/100
5/5 [==============================] - 19s 4s/step - loss: 0.4513 - accuracy: 0.8188 - val_loss: 0.5408 - val_accuracy: 0.6800
Epoch 5/100
5/5 [==============================] - 17s 3s/step - loss: 0.5092 - accuracy: 0.7785 - val_loss: 0.4151 - val_accuracy: 0.8400
Epoch 6/100
5/5 [==============================] - 18s 4s/step - loss: 0.4571 - accuracy: 0.8054 - val_loss: 0.6932 - val_accuracy: 0.6200
Epoch 7/100
5/5 [==============================] - 18s 4s/step - loss: 0.6508 - accuracy: 0.6711 - val_loss: 0.5291 - val_accuracy: 0.7400


In [39]:
print(model.history.history)

{'loss': [0.5906243764314075, 0.5354454469360761, 0.5483702971883442, 0.4510720722227289, 0.5034363777845497, 0.4568101643715929, 0.6378331368401546], 'accuracy': [0.67785233, 0.7516779, 0.7246377, 0.8187919, 0.7785235, 0.80536914, 0.67114097], 'val_loss': [0.5696409642696381, 0.5544193685054779, 0.5481173247098923, 0.5408102869987488, 0.415128618478775, 0.6931792795658112, 0.529105007648468], 'val_accuracy': [0.64, 0.66, 0.72, 0.68, 0.84, 0.62, 0.74]}
